In [66]:
import requests
import json
import pandas as pd
import numpy as np

In [86]:
RESUME_URL = 'https://jennjwang.pythonanywhere.com'
CAND_URL = 'https://heonlee.pythonanywhere.com'

# function to call api with dataset 
def call_api_scorer(candidate_data, url):

    # header for api call
    headers = {'Content-Type': 'application/json'}

    # repsonse from api 
    response = requests.post(url, data=candidate_data, headers=headers)

    return response

# function to append api responses to data 
def add_response(candidate_data, response, final=True):

    # get api prediction as json 
    predictions = json.loads(json.loads(response.text)['prediction'])

    # for each candidate, add prediction label 
    for candidate, pred in zip(candidate_data, predictions):
        if final:
            candidate['Prediction'] = pred['prediction']
        else:
            candidate['Resume score'] = pred['score']
    
    return candidate_data

In [102]:
def main():
    # Read the JSON data file
    with open('../data/candidate_samples.json', 'r') as file:
        candidate_data = json.load(file)
    
    api_data = json.dumps(candidate_data)

    # convert data to json 
    # updated_data = json.dumps(data)

    # call scorer api to get candidate resume scores 
    scorer_resp = call_api_scorer(api_data,RESUME_URL)

    # append scores to data 
    candidate_data = add_response(candidate_data, scorer_resp, final = False)

    api_data = json.dumps(candidate_data)

    # call candidate scorer to get final preds
    final_preds = call_api_scorer(api_data, CAND_URL)

    # add final predictions to dataset 
    candidate_data = add_response(candidate_data, final_preds)

    return candidate_data

# predictions for sampled candidates 
candidate_preds = main()

In [103]:
candidate_preds

[{'Applicant ID': 1,
  'School Name': 'Brown University',
  'GPA': '3.0',
  'Degree': 'Phd',
  'Location': 'New York City',
  'Gender': 'F',
  'Veteran status': '1',
  'Work authorization': '0',
  'Disability': '1',
  'Ethnicity': '1',
  'Role 1': 'ML Engineer',
  'Start 1': '4/17',
  'End 1': '11/21',
  'Role 2': 'Chef',
  'Start 2': '5/13',
  'End 2': '3/14',
  'Role 3': 'Junior SWE',
  'Start 3': '12/11',
  'End 3': '9/13',
  'Resume score': '5.15',
  'Prediction': '0'},
 {'Applicant ID': 2,
  'School Name': 'Columbia University',
  'GPA': '4.0',
  'Degree': 'Bachelors',
  'Location': 'Los Angeles',
  'Gender': 'N/A',
  'Veteran status': 'N/A',
  'Work authorization': '1',
  'Disability': '0',
  'Ethnicity': '3',
  'Role 1': 'N/A',
  'Start 1': 'N/A',
  'End 1': 'N/A',
  'Role 2': 'N/A',
  'Start 2': 'N/A',
  'End 2': 'N/A',
  'Role 3': 'N/A',
  'Start 3': 'N/A',
  'End 3': 'N/A',
  'Resume score': '2.63',
  'Prediction': '0'},
 {'Applicant ID': 3,
  'School Name': 'Brown University